### Problem Descripition 

In 2012, URL shortening service Bitly partnered with the US government website USA.gov to provide a feed of anonymous data gathered from users who shorten links ending with .gov or .mil.

The text file comes in JSON format and here are some keys and their description. They are only the most important ones for this task.

|key| description |
|---|-----------|
| a|Denotes information about the web browser and operating system|
| tz | time zone |
| r | URL the user come from |
| u | URL where the user headed to |
| t | Timestamp when the user start using the website in UNIX format |
| hc | Timestamp when user exit the website in UNIX format |
| cy | City from which the request intiated |
| ll | Longitude and Latitude |

In the cell, I tried to provide some helper code for better understanding and clearer vision

-**HINT**- Those lines of code may be not helping at all with your task.

In [1]:
# I will try to retrieve one instance of the file in a list of dictionaries
import json
records = [json.loads(line) for line in open('usa.gov_click_data.json')]
# Print the first occurance
records[0]

{'a': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.78 Safari/535.11',
 'c': 'US',
 'nk': 1,
 'tz': 'America/New_York',
 'gr': 'MA',
 'g': 'A6qOVH',
 'h': 'wfLQtf',
 'l': 'orofrog',
 'al': 'en-US,en;q=0.8',
 'hh': '1.usa.gov',
 'r': 'http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/wfLQtf',
 'u': 'http://www.ncbi.nlm.nih.gov/pubmed/22415991',
 't': 1331923247,
 'hc': 1331822918,
 'cy': 'Danvers',
 'll': [42.576698, -70.954903]}

## Required

Write a script can transform the JSON files to a DataFrame and commit each file to a sparete CSV file in the target directory and consider the following:

        

All CSV files must have the following columns
- web_browser
        The web browser that has requested the service
- operating_sys
        operating system that intiated this request
- from_url

        The main URL the user came from

    **note**:

    If the retrived URL was in a long format `http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/wfLQtf`

     make it appear in the file in a short format like this `www.facebook.com`
     
    
- to_url

       The same applied like `to_url`
   
- city

        The city from which the the request was sent
    
- longitude

        The longitude where the request was sent
- latitude

        The latitude where the request was sent

- time_zone
        
        The time zone that the city follow
        
- time_in

        Time when the request started
- time_out
        
        Time when the request is ended
        
        
**NOTE** :

Because that some instances of the file are incomplete, you may encouter some NaN values in your transforamtion. Make sure that the final dataframes have no NaNs at all.

In [2]:
import pandas as pd
import numpy as np

# place the json records into a pandas dataframe
df = pd.DataFrame(records)

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# The records with '_heartbeat_' were not part of the data so they were removed
df.drop(df[df['_heartbeat_'].notnull()].index, axis = 0, inplace = True)

In [ ]:
# Remove the unnecessary columns and reset the index
df.drop(columns = ['_heartbeat_', 'al', 'c', 'g', 'gr', 'h', 'hh', 'kw', 'l', 'nk'], inplace = True)
df.reset_index(drop = True, inplace = True)

#print an example of the data
df.sample(10)

## Extracting Web Browser & Operating System

In [ ]:
#look at the values in the 'a' columns to infer how to extract the browser name and OS
df['a'].value_counts()

In [ ]:
#function to extract the browser as the first word in the entry

def get_browser(text):
    browser = ''
    for alpha in text:
        if alpha.isalpha() is False:
            break
        browser = browser + alpha
    return browser

#function to extract the OS as the first attribute between brackets.
#If there are no brackets, then the OS is not mentioned in the entry.
def get_OS(text):
    OS = ''
    if '(' in text:
        for alpha in text[text.index('(')+1:]:
            if alpha.isalpha() is False:
                break
            OS = OS + alpha
        return OS
    else:
        return ''

In [ ]:
# Apply functions to get browser and OS in new columns
df['web_browser'] = df['a'].apply(get_browser)
df['operating_sys'] = df['a'].apply(get_OS)

#Drop the old column
df.drop(columns = 'a', inplace = True)

In [ ]:
#Print output from web_browser
df['web_browser'].value_counts()

In [ ]:
#Print output values for the OS
df['operating_sys'].value_counts()

## Extracting URLs

In [ ]:
# rename columns for easier handling
df.rename(columns = {'r' : 'from_url', 'u' : 'to_url'}, inplace = True)

In [ ]:
#function to extract the general url from the data
def get_url(text):
    if text == 'direct':
        return text
    
    #url starts after //
    text = text[text.index('/')+2:]
    
    #url ends with the end of the line or another /
    clean_url = text[:text.index('/')] if '/' in text else text 
    
    return clean_url

In [ ]:
#Apply function on the url column to clean them
df['from_url'] = df['from_url'].apply(get_url)
df['to_url'] = df['to_url'].apply(get_url)

#print a sample to see the output
df.sample(5)

## Extracting Latitude and Longitude

In [ ]:
# The column ll has both in a list where index 0 is the latitude and 1 is the longitude

#Extract the longitude
def get_long(text):
    if text is np.NaN:
        # If the long and lat are not provided, return NaN
        return np.NaN
    else:
        return text[1]

#Extra the latitude
def get_lat(text):
    if text is np.NaN:
        # If the long and lat are not provided, return NaN
        return np.NaN
    else:
        return text[0]

In [ ]:
#Place longitude in new column
df['longitude'] = df['ll'].apply(get_long)

#replace column ll to have only the latitude and rename
df['ll'] = df['ll'].apply(get_lat)
df.rename(columns = {'ll' : 'latitude'}, inplace = True)

#print a sample to see the output
df.sample(5)

## Extracting Date and Time from the Unix Format

In [ ]:
# Change time from UNIX format
df['hc'] = pd.to_datetime(df['hc'], unit='s').dt.tz_localize('GMT')
df['t'] = pd.to_datetime(df['t'], unit='s').dt.tz_localize('GMT')

#Note that the dates have been localized to the Greenwich timezone for consistency ONLY

In [ ]:
#Rename columns as per the request
df.rename(columns = {'t' : 'time_in', 'hc' : 'time_out'}, inplace = True)

## Cleaning up final columns and renaming

In [ ]:
df.rename(columns = {'cy' : 'city', 'tz' : 'timezone'}, inplace = True)

#Clean Time zone column to make it readable
df['timezone'] = df['timezone'].str.replace('/', ', ')
df['timezone'] = df['timezone'].str.replace('_', ' ')

#reorder columns for better readability
df = df[['web_browser', 'operating_sys',
         'from_url', 'to_url',
         'city', 'longitude', 'latitude',
         'timezone', 'time_in', 'time_out']]

#print the final cleaned and ordered dataframe
df.sample(10)